# Sistemas Inteligentes - Agentes e Ambientes / 1

## Guião Laboratorial 

(28/2/2018)

### Introdução

Neste laboratório vamos ver como podemos executar agentes nos seus ambientes. Recordando, um *agente* é uma entidade que percepciona o *ambiente* em que se encontra e, em *função* das percepções que recebe através de sensores, atua nesse ambiente recorrendo aos seus atuadores.

Temos assim três conceitos centrais:
* Agente – entidade que percepciona e atua;
* Ambiente – contexto em que o agente opera;
* Programa – concretiza a função do agente (mapeamento percepções/ações).

A implementação que vamos seguir baseia-se no código disponível em [agentes_e_ambientes.zip](https://moodle.ciencias.ulisboa.pt/pluginfile.php/107606/mod_page/content/5/agentes_e_ambientes.zip), para o qual está disponível o respectivo [diagrama de classes](https://moodle.ciencias.ulisboa.pt/pluginfile.php/107606/mod_page/content/5/classes_Agents.png).
Este código assenta na definição de duas classes principais: **Agent** e **Environment**. O zip hiperligado inclui três ficheiro: agents.py, o principal, e os auxiliares grid.py e utils.py.

Em primeiro lugar, deverá carregar estes três ficheiros para um directório pessoal.

No que se segue, vá criando, no mesmo directório onde colocou os três anteriores, ficheiros Python com o código ilustrado, com as adições que forem indicadas, e realizando as experiências sugeridas (e outras).

### Primeira parte - um agente

Considere o seguinte código:

In [9]:
from agents import *

class BlindDog(Agent):
    def eat(self, thing):
        print("Dog: Ate food at {}.".format(self.location))
            
    def drink(self, thing):
        print("Dog: Drank water at {}.".format(self.location))


O **BlindDog** é então uma subclasse de **Agent**, para a qual são definidos dois métodos, ***eat()*** e ***drink()***, que concretizam uma forma (para já) rudimentar das acções de comer e beber.

Vamos experimentar:

In [10]:
cao = BlindDog()

Vejamos os atributos do cão que acabámos de criar:

In [11]:
for a in cao.__dict__.keys() :
    print(a)
print("2")

alive
bump
holding
performance
program
2


Todos estes atributos foram herdados da classe **Agent** e definidos aquando da execução do  construtor (veja no código original). Alguns deles não são relevantes por agora, mas vejamos dois deles:

In [12]:
print(cao.alive)
print(cao.program)

True
<function Agent.__init__.<locals>.program at 0x7ffae844bd90>


Ou seja, o *cao* foi criado vivo (o atributo 'alive' é True) e o seu programa está representado pela função indicada.

Nos métodos ***eat()*** e ***drink()*** definidos nesta classe, existe uma referência a um atributo ***location***. Vejamos o seu valor:

In [13]:
print(cao.location)

AttributeError: 'BlindDog' object has no attribute 'location'

A localização do cão não está definida, pois esta só faz sentido se o cão estiver inserido num ambiente concreto.

### Segunda parte - o ambiente

Temos então que colocar o cão num ambiente em que ele se possa divertir. Vamos ao Parque!

Considere o seguinte código:

In [14]:
class Food(Thing):
    pass

class Water(Thing):
    pass

class Park(Environment):
    def percept(self, agent):
        '''prints & return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        print(things)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog pick eat the first item
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will or it with when there is no
        more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) \
                             for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


Criámos a classe **Park** como subclasse de **Environment**. 

Há dois métodos que têm que ser definidos quando se caracteriza um ambiente especializando a classe **Environment**:
* ***percept(self, agent)*** - devolve as percepções do agente no ambiente em causa;
* ***execute_action(self, agent, action)*** - altera o ambiente em causa em função da acção executada pelo agente.

Como pode ver no código fonte original, um ambiente tem dois atributos:
* ***things*** - lista de todas as coisas que existem no ambiente;
* ***agents*** - lista dos agentes que existem no ambiente.
Quando o ambiente é criado, ambas as listas estão vazias:

In [15]:
parque = Park()
print("Coisas: {}".format(parque.things))
print("Agentes:{}".format(parque.agents))

Coisas: []
Agentes:[]


Para adicionar coisas ao ambiente, a classe **Environment** disponibiliza o método ***add_thing(self, thing, location=None)***.

Este ambiente ***Park*** tem uma estrutura muito simples, sendo simplesmente linear (uma sequência de localizações contíguas, numeradas a partir de 0).

Vamos então adicionar o cão ao Parque:

In [16]:
parque.add_thing(cao)
print("Coisas: {}".format(parque.things))
print("Agentes:{}".format(parque.agents))

Coisas: [<BlindDog>]
Agentes:[<BlindDog>]


Vejamos agora em que localização ficou o cão:

In [17]:
print(cao.location)

None


Ainda não é isto que nos interessa. O método ***add_thing()*** referido acima, quando não recebe qualquer valor para a localização, recorre a um outro método da classe **Environment** para definir uma localização por omissão. É o método ***default_location()***.

#### Exercício 1
Redefina o método ***default_location()*** de modo a que, sempre que algo é adicionado a um Park sem se explicitar a localização, a sua localização por omissão seja 0 (zero). Incorpore esta alteração numa nova versão da classe, chamando-lhe **Parque2**. 

Podemos adicionar mais coisas ao Parque. Vamos adicionar comida e água.

Vamos então importar a nova versão da classe ***Park*** que incorpora a resolução do exercício anterior.

In [18]:
%reset -f
from Parque2 import *

parque = Park()

cao = BlindDog()
comida_de_cao = Food()
agua = Water()

parque.add_thing(comida_de_cao,5)
parque.add_thing(agua,7)
parque.add_thing(cao)

ModuleNotFoundError: No module named 'Parque2'

In [19]:
print(cao.location)

NameError: name 'cao' is not defined

In [20]:
print("Coisas: {}".format(parque.things))
print("Agentes:{}".format(parque.agents))

NameError: name 'parque' is not defined

#### Exercício 2
Acrescente à classe ***Park*** um método (***\__str__()***) que produza uma String com a representação do ambiente. Considere, por exemplo, os seguintes caracteres para representar as coisas deste ambiente:
* Cão -> '*'
* Água ->'A'
* Comida -> 'C'

Note que será também necessário alterar as representações em string das coisas que existem no ambiente.
Uma versão deste método permite uma visualização como a ilustrada em seguida.

Poderá definir uma representação distinta para os ambientes do tipo **Park**, mas tenha em consideração que poderão existir várias coisas numa mesma localização. Por exemplo, é possível que exista comida e água num mesmo local.

In [21]:
print(parque)

NameError: name 'parque' is not defined

Na variante acima, assume-se sempre que é mostrado o conteúdo de todas as células até à última que esteja ocupada (7, no exemplo). Sugestão: Utilize um dicionário em que as chaves são as posições.
***
***
Já temos o cão situado num ambiente, o parque, no qual vai poder desclocar-se, comer e beber. Mas, para fazer isso, temos que definir o seu comportamento recorrendo a um programa.

### Terceira parte - o programa
O comportamento de um agente é caracterizado pela definição de uma função, que deverá ser passada como argumento do construtor do agente, para ser guardada no seu atributo ***program***.

Esta função deverá ter apenas um argumento (a percepção) e devolver a correspondente acção.

O tipo destas entidades (percepção e acção) tem que ser compatível com as definições dos métodos ***percept(self,agent)*** e ***execute_action(self,agent,action)*** definidos.

Vejamos a caracterização e definição de uma função muito simples para concretizar o comportamento básico do cão.

O comportamento que pretendemos está definido na seguinte tabela: 
<table>
    <tr>
        <td><b>Percepção:</b> </td>
        <td>Sente Comida </td>
        <td>Sente Água</td>
        <td>Não sente nada</td>
   </tr>
   <tr>
       <td><b>Acção:</b> </td>
       <td>comer</td>
       <td>beber</td>
       <td>andar</td>
   </tr>
        
</table>

Este comportamento pode ser traduzido na seguinte função, respeitando o facto de que:
* percepção - é a lista das percepções (lista de coisas - objectos da classe **Thing**)
* acção - deverá ser uma das strings identificadas em ***execute_action()***

In [ ]:
def comportamento_do_cao(percepcao):
    '''Devolve uma acção em função da percepção'''
    if percepcao == [] :
        accao = 'move down'
    elif isinstance(percepcao[0],Food) :
        accao = 'eat'
    elif isinstance(percepcao[0],Water) :
        accao = 'drink'
    else :
        accao = comportamento_do_cao(percepcao[1:]) 
    return accao


Esta função processa então apenas a primeira percepção interessante que for identificada na lista fornecida. Caso não seja identificada nenhuma, determina a movimentação do agente.

Podemos agora criar cães cujo comportamento corresponda à função definida:

In [ ]:
outro_cao = BlindDog(comportamento_do_cao)
print(outro_cao)
print(outro_cao.program)

### Quarta parte - execução da simulação
De modo a simular o comportamento do cão no parque, é necessário melhorar a definição da classe **BlindDog** de modo a que os métodos correspondentes à execução das ações realizem o que o método ***execute_action()*** espera.

In [ ]:
class BlindDog(Agent):

    def movedown(self):
        self.location += 1
    
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            print("Dog: Ate food at {}.".format(self.location))
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            print("Dog: Drank water at {}.".format(self.location))
            return True
        return False

    def __str__(self) :
        return '*'


Vamos então eliminar o cao anterior e criar um cão que execute o comportamento ilustrado acima:

In [ ]:
parque.delete_thing(cao)
cao = BlindDog(comportamento_do_cao)
parque.add_thing(cao,2)
print(parque)

A execução dos agentes é feita com base no método ***step()***. Vejamos a sua definião na classe **Environment**:
```python
def step(self):
    """Run the environment for one time step. If the
    actions and exogenous changes are independent, this method will
    do.  If there are interactions between them, you'll need to
    override this method."""
    if not self.is_done():
        actions = []
        for agent in self.agents:
            if agent.alive:
                actions.append(agent.program(self.percept(agent)))
            else:
                actions.append("")
        for (agent, action) in zip(self.agents, actions):
            self.execute_action(agent, action)
        self.exogenous_change()
```

Algumas notas importantes sobre este método:
* A condição de terminação da execução é dada pelo método ***is_done()***, definido na classe **Park**. Neste exemplo, a simulação termina quando todos agentes estiverem mortos (alive == False) ou quando acabar a comida;
* Todos os agentes vivos executam uma ação em cada passo.

Vamos experimentar:

In [ ]:
parque.step()
print(parque)
parque.step()
parque.step()
print(parque)

Em cada passo, a chamada ao método ***percept()*** produz a escrita da lista das percepções do agente, que, no exemplo acima, corresponde ao facto de o agente se autopercepcionar!

De modo a correr vários passos de uma só vez, existe o método ***run(self,steps)*** que executa o número de passos dado (no máximo).

In [ ]:
parque.run(6)
print(parque)

#### Exercício 3
Altere a definição do método ***percept()*** de modo a que na lista de percepções que é escrita em cada passo apareçam as representações em string definidas para cada tipo de coisa, ou seja, por exemplo, de modo a que apareça ['C','`*`'], em vez de `[<Food>, <BlindDog>]`.

#### Exercício 4
Altere a definição da classe **BlindDog** de modo a que os cães sejam criados com nome. O nome dado será usado como representação do cão, facilitando assim a identificação dos agentes, nas situações em que existem vários em simultâneo no ambiente. Altere os métodos onde este aspecto é relevante.

Por exemplo, a sequência de instruções:
```python
    parque = Park()
    dog1 = BlindDog("Bobi",comportamento_do_cao)
    dog2 = BlindDog("Milou",comportamento_do_cao)
    dogfood = Food()
    water = Water()
    parque.add_thing(dog1, 2)
    parque.add_thing(dog2, 6)
    parque.add_thing(dogfood, 5)
    parque.add_thing(water, 7)
    print(parque)
```

Deverá produzir o seguinte output:
```
0: 
1: 
2: /Bobi
3: 
4: 
5: /C
6: /Milou
7: /A
```

#### Exercício 5 - um ambiente 2D
Defina agora um parque a duas dimensões recorrendo à classe **XYEnvironment** (que estende a **Environment**).
```python
class Park2D(XYEnvironment) :
    ...

parque2 = Park2D(6,6) # cria parque de dimensão 6x6
```

Por exemplo, a sequência de instruções:
```python
    parque = Park2D(6,6)
    dog1 = BlindDog("*",comportamento_do_cao)
    dog2 = BlindDog("@",comportamento_do_cao)
    dogfood = Food()
    water = Water()
    parque.add_thing(dog1, (1,2))
    parque.add_thing(dog2, (3,4))
    parque.add_thing(dogfood, (4,3))
    parque.add_thing(water, (4,1))
    print(parque)
```

Deverá produzir um output semelhante ao seguinte:
```
     0   1   2   3   4   5 
  0                   
  1             /A    
  2    /*             
  3             /C    
  4          /@       
  5                   
```

Algumas observações para ajudar a resolução:
* Para além dos atributos ***things*** e ***agents***, a classe **EnvirnomentXY** tem também um conjunto de atributos relacionados com a dimensão do espaço (***width***, ***height***, ***x_start***, ***x_end***,***y_start***, ***y_end***).
* Cada localização é definida por um par (X,Y). No exemplo acima, as localizações válidas do *parque2* vão de (0,0) a (5,5).
* O método booleano ***is_inbounds()*** permite verificar se uma dada coordenada é válida.

Muitos dos métodos definidos para **Park** podem ser aproveitados. Alguns têm que ser adaptados. Assim:
* Na classe **Park2D**:
    * Altere o método ***execute_action()*** de modo a lidar com as diferentes acções de movimentação.
    * Altere o método ***\__str__()***
* Na classe **Dog**:
    * Defina uma forma de representar as acções de movimentação nos quatro sentidos principais.
* Defina funções que traduzam comportamentos diferentes relativamente à movimentação, por exemplo:
    * movimentação aleatória
    * movimentação em função de visão limitada de uma unidade, deslocando-se para comida (ou água)
    